<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Max-Sharpe-Ratio-Portfolio" data-toc-modified-id="Max-Sharpe-Ratio-Portfolio-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Max Sharpe Ratio Portfolio</a></span><ul class="toc-item"><li><span><a href="#Librerías" data-toc-modified-id="Librerías-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Librerías</a></span></li><li><span><a href="#Cargar-datos" data-toc-modified-id="Cargar-datos-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Cargar datos</a></span></li><li><span><a href="#Encontrar-la-cartera-con-máximo-Sharpe-Ratio" data-toc-modified-id="Encontrar-la-cartera-con-máximo-Sharpe-Ratio-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Encontrar la cartera con máximo Sharpe-Ratio</a></span></li></ul></li></ul></div>

## Max Sharpe Ratio Portfolio

La frontera eficiente puede ser trazada/calculada a partir de un conjunto dado de rendimientos esperados y la matriz de covarianza. Una vez obtenida, es posible localizar el punto de la frontera eficiente que más interesa, que es la cartera de tangencia o la cartera de Max Sharpe Ratio.

### Librerías

In [17]:
import pandas as pd
from modules import edhec_risk_kit as erk
import plotly 
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import minimize

# Modo desarrollo del módulo edhec_risk_kit
%matplotlib inline
%load_ext autoreload
%autoreload 2
plotly.offline.init_notebook_mode(connected = True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Cargar datos

In [18]:
df_ind = erk.get_ind_returns(filename='../data/ind30_m_vw_rets.csv')
df_er = erk.annualize_rets(df_ind['1996':'2000'], 12)
df_cov = df_ind['1996':'2000'].cov()

### Encontrar la cartera con máximo Sharpe-Ratio

Ya sabemos cómo identificar los puntos de la curva si se nos da una tasa de rendimiento objetivo. En lugar de minimizar la volatilidad basado en un objetivo de rentabilidad, a continuación, se tratará de encontrar ese punto de la curva que maximiza el ratio de Sharpe, dada la tasa libre de riesgo.

In [19]:
rfr = 0.1 # Risk Free Rate

# Calcular pesos óptimos identificando la cartera sobre la frontera eficiente
# con mayor Sharpe Ratio (rendimiento ajustado al riesgo por unidad de voltailidad)
w_msr = erk.msr(risk_free_rate=rfr, expected_returns=df_er, covmat=df_cov).round(4)
ret_msr = erk.portfolio_return(w_msr, df_er)
vol_msr = erk.portfolio_volatility(w_msr, df_cov)

df_optimal_portfolio = pd.DataFrame(columns=['Assets', 'Weights'])
df_optimal_portfolio['Assets'] = pd.DataFrame(df_er).index.to_list()
df_optimal_portfolio['Weights'] = w_msr
df_optimal_portfolio.sort_values('Weights', ascending=False, inplace=True)
df_optimal_portfolio.reset_index(drop=True, inplace=True)

print('Portfolio return = {:.2f}%'.format(ret_msr*100))
print('Portfolio volatility = {:.2f}%'.format(vol_msr*100))
df_optimal_portfolio

Portfolio return = 26.47%
Portfolio volatility = 4.57%


,Assets,Weights
0,Hlth,0.4560
1,ElcEq,0.2338
2,Coal,0.1387
3,Util,0.0894
4,Rtail,0.0666
5,BusEq,0.0155
6,Food,0.0000
7,Mines,0.0000
8,Fin,0.0000
9,Meals,0.0000


In [20]:
# Visualización
erk.plot_ef(n_points = 20, 
            expected_returns = df_er, 
            covmat = df_cov, 
            title='Efficient Frontier Portfolio Maximum Sharpe Ratio', 
            show_cml=True, 
            risk_free_rate=0.1)